In [ ]:
import os
import json
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

from sklearn.model_selection import train_test_split


In [ ]:
# User
file_path = os.path.join('accounts/fixtures/accounts', 'fake_user.json')
with open(file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

df_data = {
    'birth': [],
    'gender': [],
    'crdt_grad': [],
    'salary': [],
    'm_consumption':[],
    'asset':[],
    'real_estate':[],
    'invest_tendency':[],
    'pk':[],
    'pro_cnt':[]
}

for i in data:
    a = i['fields']['product']
    for pro in a:
        df_data['birth'].append(i['fields']['birth'])
        df_data['gender'].append(i['fields']['gender'])
        df_data['crdt_grad'].append(i['fields']['crdt_grad'])
        df_data['salary'].append(i['fields']['salary'])
        df_data['m_consumption'].append(i['fields']['m_consumption'])
        df_data['asset'].append(i['fields']['asset'])
        df_data['real_estate'].append(i['fields']['real_estate'])
        df_data['invest_tendency'].append(i['fields']['invest_tendency'])
        df_data['pk'].append(pro)
        df_data['pro_cnt'].append(len(a))
user = pd.DataFrame(df_data)

# Deposit products
file_path = os.path.join('savedata/fixtures/savedata', 'deposit_products.json')
with open(file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
df_data = {
    'pk': [],               # pk
    'company': [],        # 금융회사코드
    'product_number': [],      # 금융상품코드
    'join_deny': [],        # 가입 제한
    'join_member': [],      # 가입 대상
    'join_way': [],         # 가입 방법
    'type':[],      # 예금/적금
    'join_way_cnt':[]
}

for i in data:
    a = i['fields']['join_way'].split(',')
    for join in a:
        df_data['pk'].append(i['pk'])
        df_data['company'].append(i['fields']['fin_co_no'])
        df_data['product_number'].append(i['fields']['fin_prdt_cd'])
        df_data['join_deny'].append(i['fields']['join_deny'])
        df_data['join_member'].append(i['fields']['join_member'])
        df_data['join_way'].append(join)
        df_data['type'].append(i['fields']['product_type'])
        df_data['join_way_cnt'].append(len(a))
products = pd.DataFrame(df_data)

# Deposit options
file_path = os.path.join('savedata/fixtures/savedata', 'deposit_options.json')
with open(file_path, 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)
df_data = {
    'company': [],
    'product_number': [],
    'intr_rate_type_nm': [],
    'intr_rate': [],
    'intr_rate2': [],
    'save_trm':[],
    'rsrv_type':[],
}

for i in data:
    df_data['company'].append(i['fields']['fin_co_no'])
    df_data['product_number'].append(i['fields']['fin_prdt_cd'])
    df_data['intr_rate_type_nm'].append(i['fields']['intr_rate_type_nm'])
    df_data['intr_rate'].append(i['fields']['intr_rate'])
    df_data['intr_rate2'].append(i['fields']['intr_rate2'])
    df_data['save_trm'].append(i['fields']['save_trm'])
    df_data['rsrv_type'].append(i['fields']['rsrv_type'])
options = pd.DataFrame(df_data)

## 전처리

In [ ]:
display(user.head())
display(user.isnull().sum())
print()
display(products.head())
display(products.isnull().sum())
print()
display(options.head())
display(options.isnull().sum())

,birth,gender,crdt_grad,salary,m_consumption,asset,real_estate,invest_tendency,pk,pro_cnt
0,1955-11-19,1,874,4412,851,19667,1,2,184,3
1,1955-11-19,1,874,4412,851,19667,1,2,379,3
2,1955-11-19,1,874,4412,851,19667,1,2,400,3
3,1992-12-12,1,361,20297,266,29320,0,3,325,2
4,1992-12-12,1,361,20297,266,29320,0,3,429,2


birth              0
gender             0
crdt_grad          0
salary             0
m_consumption      0
asset              0
real_estate        0
invest_tendency    0
pk                 0
pro_cnt            0
dtype: int64

,pk,company,product_number,join_deny,join_member,join_way,type,join_way_cnt
0,1,0010001,WR0001B,1,실명의 개인,인터넷,예금,3
1,1,0010001,WR0001B,1,실명의 개인,스마트폰,예금,3
2,1,0010001,WR0001B,1,실명의 개인,전화(텔레뱅킹),예금,3
3,2,0010002,00320342,1,개인(개인사업자 포함),인터넷,예금,2
4,2,0010002,00320342,1,개인(개인사업자 포함),스마트폰,예금,2


pk                0
company           0
product_number    0
join_deny         0
join_member       0
join_way          0
type              0
join_way_cnt      0
dtype: int64

,company,product_number,intr_rate_type_nm,intr_rate,intr_rate2,save_trm,rsrv_type
0,0010001,WR0001B,단리,3.00,3.00,1,None
1,0010001,WR0001B,단리,3.50,3.50,3,None
2,0010001,WR0001B,단리,3.50,3.50,6,None
3,0010001,WR0001B,단리,3.55,3.55,12,None
4,0010001,WR0001B,단리,3.00,3.00,24,None


company                 0
product_number          0
intr_rate_type_nm       0
intr_rate               0
intr_rate2              0
save_trm                0
rsrv_type            2782
dtype: int64

In [ ]:
pro = products.merge(options)
pro.head(5)

,pk,company,product_number,join_deny,join_member,join_way,type,join_way_cnt,intr_rate_type_nm,intr_rate,intr_rate2,save_trm,rsrv_type
0,1,0010001,WR0001B,1,실명의 개인,인터넷,예금,3,단리,3.00,3.00,1,None
1,1,0010001,WR0001B,1,실명의 개인,인터넷,예금,3,단리,3.50,3.50,3,None
2,1,0010001,WR0001B,1,실명의 개인,인터넷,예금,3,단리,3.50,3.50,6,None
3,1,0010001,WR0001B,1,실명의 개인,인터넷,예금,3,단리,3.55,3.55,12,None
4,1,0010001,WR0001B,1,실명의 개인,인터넷,예금,3,단리,3.00,3.00,24,None


In [ ]:
df = user.merge(pro, on="pk")
df

,birth,gender,crdt_grad,salary,m_consumption,asset,real_estate,invest_tendency,pk,pro_cnt,...,join_deny,join_member,join_way,type,join_way_cnt,intr_rate_type_nm,intr_rate,intr_rate2,save_trm,rsrv_type
0,1955-11-19,1,874,4412,851,19667,1,2,184,3,...,1,내·외국인 대상,인터넷,예금,1,단리,1.2,1.2,1,None
1,1955-11-19,1,874,4412,851,19667,1,2,184,3,...,1,내·외국인 대상,인터넷,예금,1,단리,3.0,3.0,3,None
2,1955-11-19,1,874,4412,851,19667,1,2,184,3,...,1,내·외국인 대상,인터넷,예금,1,단리,3.6,3.6,6,None
3,1955-11-19,1,874,4412,851,19667,1,2,184,3,...,1,내·외국인 대상,인터넷,예금,1,단리,3.9,3.9,12,None
4,1955-11-19,1,874,4412,851,19667,1,2,184,3,...,1,내·외국인 대상,인터넷,예금,1,단리,3.0,3.0,24,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25024,1963-07-27,1,974,18429,65,93292,1,4,524,1,...,1,제한없음,영업점,적금,1,단리,1.7,1.7,24,S
25025,1963-07-27,1,974,18429,65,93292,1,4,524,1,...,1,제한없음,영업점,적금,1,단리,2.7,2.7,12,S
25026,1963-07-27,1,974,18429,65,93292,1,4,524,1,...,1,제한없음,영업점,적금,1,단리,2.0,2.0,6,S
25027,1961-11-04,0,86,6255,403,-4032,1,5,405,5,...,1,제한없음,스마트폰,예금,1,단리,3.6,3.6,36,None


In [14]:
df.nunique()

birth                978
gender                 2
crdt_grad            635
salary               977
m_consumption        630
asset                996
real_estate            2
invest_tendency        6
pk                   730
pro_cnt                5
company               97
product_number       476
join_deny              2
join_member           98
join_way               7
type                   2
join_way_cnt           4
intr_rate_type_nm      2
intr_rate            116
intr_rate2           139
save_trm               6
rsrv_type              2
dtype: int64

In [15]:
# 예금 상품 type 결측치 처리
df.loc[df['rsrv_type'].isna(), 'rsrv_type'] = 'N'

In [16]:
df.groupby(['type','rsrv_type'])[['rsrv_type']].count()

rsrv_type
type rsrv_type           
예금   F                 36
     N              16073
     S                 48
적금   F               1954
     S               6918

In [17]:
# birth datetime 변환
df['birth'] = pd.to_datetime(df['birth'])

In [18]:
del df['company']
del df['product_number']

## Feature 생성

In [19]:
df

,birth,gender,crdt_grad,salary,m_consumption,asset,real_estate,invest_tendency,pk,pro_cnt,join_deny,join_member,join_way,type,join_way_cnt,intr_rate_type_nm,intr_rate,intr_rate2,save_trm,rsrv_type
0,1955-11-19,1,874,4412,851,19667,1,2,184,3,1,내·외국인 대상,인터넷,예금,1,단리,1.2,1.2,1,N
1,1955-11-19,1,874,4412,851,19667,1,2,184,3,1,내·외국인 대상,인터넷,예금,1,단리,3.0,3.0,3,N
2,1955-11-19,1,874,4412,851,19667,1,2,184,3,1,내·외국인 대상,인터넷,예금,1,단리,3.6,3.6,6,N
3,1955-11-19,1,874,4412,851,19667,1,2,184,3,1,내·외국인 대상,인터넷,예금,1,단리,3.9,3.9,12,N
4,1955-11-19,1,874,4412,851,19667,1,2,184,3,1,내·외국인 대상,인터넷,예금,1,단리,3.0,3.0,24,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25024,1963-07-27,1,974,18429,65,93292,1,4,524,1,1,제한없음,영업점,적금,1,단리,1.7,1.7,24,S
25025,1963-07-27,1,974,18429,65,93292,1,4,524,1,1,제한없음,영업점,적금,1,단리,2.7,2.7,12,S
25026,1963-07-27,1,974,18429,65,93292,1,4,524,1,1,제한없음,영업점,적금,1,단리,2.0,2.0,6,S
25027,1961-11-04,0,86,6255,403,-4032,1,5,405,5,1,제한없음,스마트폰,예금,1,단리,3.6,3.6,36,N


In [ ]:
# 나이
df['age'] = df['birth'].apply(lambda x: datetime.now().year-x.year+1)

In [ ]:
# join_way의 갯수가 많은 순대로 가중치
pd.DataFrame(df['join_way'].value_counts())
df['hot_join'] = df['join_way'].apply(lambda x: 7 if x == "스마트폰"
                                        else 6 if x == "영업점"
                                        else 5 if x =="인터넷"
                                        else 4 if x == "전화(텔레뱅킹)"
                                        else 3 if x == "기타"
                                        else 2 if x == "모집인"
                                        else 1)

In [ ]:
pro_df = df.groupby('pk')[['crdt_grad','salary','m_consumption','asset','intr_rate','intr_rate2','save_trm']].mean().reset_index()
pro_df.rename(columns=lambda x: f"{x}_avg" if x != "pk" else x, inplace=True)

In [ ]:
# 소비 상황
df.loc[df['salary'] < df['m_consumption']*12, "consumption"] = "과소비"
df.loc[df['salary'] == df['m_consumption']*12, "consumption"] = "위험"
df.loc[df['salary'] > df['m_consumption']*12, "consumption"] = "적정"

In [ ]:
df = df.merge(pro_df)

### 모델링

In [ ]:
df['birth'] = df['birth'].astype('str')
df['gender'] = df['gender'].astype('category')
df['real_estate'] = df['real_estate'].astype('category')
df['invest_tendency'] = df['invest_tendency'].astype('category')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25029 entries, 0 to 25028
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   birth              25029 non-null  object  
 1   gender             25029 non-null  category
 2   crdt_grad          25029 non-null  int64   
 3   salary             25029 non-null  int64   
 4   m_consumption      25029 non-null  int64   
 5   asset              25029 non-null  int64   
 6   real_estate        25029 non-null  category
 7   invest_tendency    25029 non-null  category
 8   pk                 25029 non-null  int64   
 9   pro_cnt            25029 non-null  int64   
 10  join_deny          25029 non-null  int64   
 11  join_member        25029 non-null  object  
 12  join_way           25029 non-null  object  
 13  type               25029 non-null  object  
 14  join_way_cnt       25029 non-null  int64   
 15  intr_rate_type_nm  25029 non-null  object  
 16  intr

In [ ]:
# 범주형 변수와 수치형 변수를 분리

cat_df = df.select_dtypes(include=['object','category']).columns.to_list()
num_df = df.select_dtypes(exclude=['object','category']).columns.to_list()

In [ ]:
# 범주형 변수에 One-Hot-Encoding 후 수치형 변수와 병합

if len(cat_df) > 0:
    df = pd.concat([df[num_df], pd.get_dummies(df[cat_df])], axis=1)
else:
    df = df[num_df]

df

,crdt_grad,salary,m_consumption,asset,pk,pro_cnt,join_deny,join_way_cnt,intr_rate,intr_rate2,...,join_way_정보 없음,type_예금,type_적금,intr_rate_type_nm_단리,intr_rate_type_nm_복리,rsrv_type_F,rsrv_type_N,rsrv_type_S,consumption_과소비,consumption_적정
0,874,4412,851,19667,184,3,1,1,1.2,1.2,...,False,True,False,True,False,False,True,False,True,False
1,874,4412,851,19667,184,3,1,1,3.0,3.0,...,False,True,False,True,False,False,True,False,True,False
2,874,4412,851,19667,184,3,1,1,3.6,3.6,...,False,True,False,True,False,False,True,False,True,False
3,874,4412,851,19667,184,3,1,1,3.9,3.9,...,False,True,False,True,False,False,True,False,True,False
4,874,4412,851,19667,184,3,1,1,3.0,3.0,...,False,True,False,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25024,974,18429,65,93292,524,1,1,1,1.7,1.7,...,False,False,True,True,False,False,False,True,False,True
25025,974,18429,65,93292,524,1,1,1,2.7,2.7,...,False,False,True,True,False,False,False,True,False,True
25026,974,18429,65,93292,524,1,1,1,2.0,2.0,...,False,False,True,True,False,False,False,True,False,True
25027,86,6255,403,-4032,405,5,1,1,3.6,3.6,...,False,True,False,True,False,False,True,False,False,True


In [ ]:
products['pk'].count()

1109

In [ ]:
X = df.drop(columns=['pk'])
y = df['pk']

# 학습 세트와 테스트 세트로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 로지스틱 회귀 모델 생성
model = LogisticRegression()

# 모델 학습
model.fit(X_train, y_train)

# 테스트 세트에 대한 예측
y_pred = model.predict(X_test)

# 모델 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9910107870555334
Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        22
           2       0.93      1.00      0.96        13
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        13
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         4
           7       1.00      1.00      1.00         7
           8       1.00      1.00      1.00        16
           9       1.00      1.00      1.00         4
          10       1.00      1.00      1.00         2
          12       1.00      1.00      1.00         2
          13       1.00      1.00      1.00        19
          16       1.00      1.00      1.00         1
          17       1.00      1.00      1.00         7
          18       1.00      1.00      1.00         6
          19       1.00      1.00      1.00         5
          20       1.00      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [ ]:
import joblib
joblib.dump(model, 'recommend_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']